In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
from collections import defaultdict
from collections import Counter

import matplotlib.pyplot as plt

In [2]:
import psutil
psutil.cpu_count()
p = psutil.Process()
p.cpu_affinity([0,1,2,3,4,5,6,7,8,9])

In [3]:
os.chdir(r'H:\all\RL_Shrinkage_2024')
from helpers import helper_functions as hf


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\dkostovic\.conda\envs\FTSE_data_analysis_v2\lib\site-packages\traitlets\con

In [4]:
import glob

path = r'H:\all\RL_Shrinkage_2024\ONE_YR\NonLinear_Shrinkage\transformed_qis_eigenvalues'
extension = 'csv'
os.chdir(path)
filenames = glob.glob('*.{}'.format(extension))
print(filenames)

['cov2para_evs_df.csv', 'cov2para_fixed04_evs_df.csv', 'cov2para_plus02_evs_df.csv', 'qis_evs_df.csv', 'qis_evs_df_p100.csv', 'qis_evs_df_p225.csv', 'qis_evs_df_p30.csv', 'qis_evs_df_p50.csv', 'qis_evs_df_p500.csv', 'qis_evs_exp_05.csv', 'qis_evs_exp_2.csv', 'sample_evs_df.csv', 'sample_evs_df_p100.csv', 'sample_evs_df_p225.csv', 'sample_evs_df_p30.csv', 'sample_evs_df_p50.csv', 'sample_evs_df_p500.csv']


In [5]:
def get_ev_intensities(evs, sample_evs):
    intensities = evs.copy()
    if sample_evs.shape[1] == 500:
        intensities.iloc[:, -251:] = evs.iloc[:, -251:].values / sample_evs.iloc[:, -251:].values
    else:
        intensities = evs.values / sample_evs.values
    return intensities

def plot_last_x_intensities_across_time(intensities_dfs, modelnames, num_evs, return_df=False):
    if len(modelnames) > 1:
        dfs_list = [intensities_dfs[M].iloc[:, -num_evs:].mean(axis=1) for M in modelnames]
        df = pd.concat(dfs_list, axis=1)
        df.columns = modelnames
    else:
        df = intensities_dfs[modelnames[0]].iloc[:, -num_evs:].mean(axis=1)
        df.name = modelnames[0]
    fig = px.line(
        df,
        title = f"Plot of Average EV(Model) / EV(Sample) Across Time",
         labels={'value':f'Mean of EV(Model) / EV(Sample)'},
        height=700
    )
    fig.show()

def plot_last_x_intensities_across_assets(intensities_dfs, modelnames, num_evs, return_df=False):
    if len(modelnames) > 1:
        dfs_list = [intensities_dfs[M].iloc[:, -num_evs:].mean() for M in modelnames]
        df = pd.concat(dfs_list, axis=1)
        df.columns = modelnames
    else:
        df = intensities_dfs[modelnames[0]].iloc[:, -num_evs:].mean()
        df.name = modelnames[0]
        
    fig = px.line(
        df,
        title = f"Plot of Average EV(Model) / EV(Sample) Across Assets",
         labels={'value':f'Mean of EV(Model) / EV(Sample)', 'index': "Eigenvalue"},
        height=700
    )
    fig.show()


def plot_last_x_eigenvalues_across_time(eigenvalues_dfs, modelnames, ev_range, return_df=False):
    if len(modelnames) > 1:
        dfs_list = [eigenvalues_dfs[M].iloc[:, ev_range[0]:ev_range[1]].mean(axis=1) for M in modelnames]
        df = pd.concat(dfs_list, axis=1)
        df.columns = modelnames
    else:
        df = eigenvalues_dfs[modelnames[0]].iloc[:, ev_range[0]:ev_range[1]].mean(axis=1)
        df.name = modelnames[0]
    fig = px.line(
        df,
        title = f"Plot of Average EV(Model) Across Time",
         labels={'value':f'Mean of EV(Model) '},
        height=700
    )
    fig.show()

def plot_last_x_eigenvalues_across_assets(eigenvalues_dfs, modelnames, ev_range, return_df=False):
    if len(modelnames) > 1:
        dfs_list = [eigenvalues_dfs[M].iloc[:, ev_range[0]:ev_range[1]].mean() for M in modelnames]
        df = pd.concat(dfs_list, axis=1)
        df.columns = modelnames
    else:
        df = eigenvalues_dfs[modelnames[0]].iloc[:, ev_range[0]:ev_range[1]].mean()
        df.name = modelnames[0]
        
    fig = px.line(
        df,
        title = f"Plot of Average EV(Model) Across Assets",
         labels={'value':f'Mean of EV(Model)', 'index': "Eigenvalue"},
        height=700
    )
    fig.show()


def get_rawres(eigenvalue_dict, modelnames: list):
    tmp_res = defaultdict(list)
    tmp_rawres = defaultdict(list)
    for idx in range(0, permnos.shape[0]):
        try:
            past_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx - 21 * 12 * 1: idx + add_idx, :]
            past_ret_mat = past_ret_mat.sub(past_ret_mat.mean())
            past_ret_mat = past_ret_mat.fillna(0)
            fut_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + add_idx: idx + add_idx + 21, :]
        except:
            print("Some Error..")
            
        N, p = past_ret_mat.shape
        sample = pd.DataFrame(np.matmul(past_ret_mat.T.to_numpy(), past_ret_mat.to_numpy())) / (N - 1)
        lambda1, u = np.linalg.eigh(sample)
        lambda1 = lambda1.real.clip(min=0)
        dfu = pd.DataFrame(u,columns=lambda1)
        dfu.sort_index(axis=1,inplace = True)
        temp1 = dfu.to_numpy()
        temp3 = dfu.T.to_numpy().conjugate()

        for cur_modelname in modelnames:
            qis = eigenvalue_dict[cur_modelname].iloc[idx, :]
            temp2 = np.diag(qis)
            sigmahat = pd.DataFrame(np.matmul(np.matmul(temp1, temp2), temp3))
            try:
                weights = hf.calc_global_min_variance_pf(sigmahat)
            except:
                print("Some Other Error..")
            # store results
            tmp_res[cur_modelname].append(np.std(fut_ret_mat @ weights, ddof=1) * np.sqrt(252) * 100)
            if idx % 21 == 0:
                tmp_rawres[cur_modelname] += list(fut_ret_mat @ weights)

        if idx % 250 == 0:
            print(f"done {idx} out of {permnos.shape[0]}")

    return tmp_rawres, tmp_res
    

In [6]:
# LOAD EIGENVALUES
PF_SIZE = 30

model_names = ["qis",  "sample"]
evs_dfs = {}
qis_evs_path = r"H:\all\RL_Shrinkage_2024\ONE_YR\NonLinear_Shrinkage\transformed_qis_eigenvalues"
evs_dfs["qis"] = pd.read_csv(qis_evs_path + f'\\qis_evs_df_p{PF_SIZE}.csv', index_col=0)
evs_dfs["sample"] = pd.read_csv(qis_evs_path + f'\\sample_evs_df_p{PF_SIZE}.csv', index_col=0)

base_folder_path = r'H:\\all\\RL_Shrinkage_2024'
permnos = pd.read_pickle(
    fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\permnos_1Y_p{PF_SIZE}.pickle")
'''
intensities_dfs = {}
for k, v in evs_dfs.items():
    v.index = pd.to_datetime(permnos.index, format="%Y%m%d")
    intensities_dfs[k] = get_ev_intensities(v, evs_dfs["sample"])
'''

'\nintensities_dfs = {}\nfor k, v in evs_dfs.items():\n    v.index = pd.to_datetime(permnos.index, format="%Y%m%d")\n    intensities_dfs[k] = get_ev_intensities(v, evs_dfs["sample"])\n'

## Get eigenvalues via new formula

In [7]:
def get_lambda_qis_nl(qis_ev, sample_ev, int_of_int):
    a = ( (qis_ev/sample_ev - 1)*int_of_int + 1) * sample_ev
    return a


def get_new_eigenvalues(qis_eigenvalues, sample_eigenvalues, intensity_of_intensity):
    intens_df = qis_eigenvalues / sample_eigenvalues
    if qis_eigenvalues.shape[1] == 500:  # only 1 year case ofcourse
        intens_df = intens_df.iloc[:, -251:]


     
    right_rotation_idx = np.argmin( np.abs(intens_df-1) , axis=1)
    left_rotation_idx = right_rotation_idx - 1
    rotation_points = 0.5 * np.diag(intens_df.iloc[:, left_rotation_idx]) + 0.5 * np.diag(intens_df.iloc[:, right_rotation_idx])


def get_new_eigenvalues(qis_eigenvalues, sample_eigenvalues, intensity_of_intensity):
    intensity = qis_eigenvalues.copy()
    if qis_eigenvalues.shape[1] == 500:
        intensity.iloc[: , -251:] = qis_eigenvalues.iloc[: , -251:] / sample_eigenvalues.iloc[: , -251:]
    else:
        intensity = qis_eigenvalues / sample_eigenvalues
    intensity_delta = intensity - 1
    intensity_delta_new = intensity_delta * intensity_of_intensity
    intensity_new = intensity_delta_new + 1
    qis_evs_new = intensity_new * sample_eigenvalues
    if qis_eigenvalues.shape[1] == 500:
        qis_evs_new.iloc[: , 0:250] = intensity_of_intensity * qis_eigenvalues.iloc[: , 0:250]

    # get Rotation Points
    intens_df = qis_eigenvalues / sample_eigenvalues
    if qis_eigenvalues.shape[1] == 500:  # only 1 year case ofcourse
        intens_df = intens_df.fillna(0)
    right_rotation_idx = np.argmin( np.abs(intens_df-1) , axis=1)
    left_rotation_idx = right_rotation_idx - 1
    rotation_point_sample_evs = 0.5 * np.diag(sample_eigenvalues.iloc[:, left_rotation_idx]) + 0.5 * np.diag(sample_eigenvalues.iloc[:, right_rotation_idx])
    rotation_points = 0.5 * np.diag(intens_df.iloc[:, left_rotation_idx]) + 0.5 * np.diag(intens_df.iloc[:, right_rotation_idx])
    rotation_points = rotation_points * rotation_point_sample_evs

    # Force all eigenvalues to be in decreasing order (from the largest one)
    idx_is_increasing = qis_evs_new.diff(axis=1).fillna(0) < 0
    qis_evs_new[idx_is_increasing] = np.nan
    qis_evs_new = qis_evs_new.bfill(axis=1)

    # check if values left (right) of rotation point are smaller (larger) than rotation point
    # check: is idx < left and value_at_idx > value_at_left --> then np.nan and ffill it for left (and bfill for right)
    for i in range(qis_evs_new.shape[0]):
        tmp = qis_evs_new.iloc[i, :].copy()
        left_bool = tmp[0:left_rotation_idx[i]] > rotation_points[i]
        right_bool = tmp[left_rotation_idx[i]:] < rotation_points[i]
        # change those evs on right that are smaller than rotation point
        tmp_right = tmp[left_rotation_idx[i]:]
        tmp_right[right_bool] = np.nan
        tmp_right.bfill(inplace=True) 
        tmp_right.ffill(inplace=True)  # in case the largest ev is NAN, we need to additionally ffill
        # change those evs on left that are LARGER than rotation point
        tmp_left = tmp[0:left_rotation_idx[i]]
        tmp_left[left_bool] = np.nan
        tmp_left.ffill(inplace=True) 
        tmp_left.bfill(inplace=True) # in case the smallest ev is NAN, we need to additionally bfill
        tmp_left.fillna(rotation_points[i], inplace=True)  # in case all eigenvalues left of rotation point are larger than rotation point
        # change qis_evs_new row
        qis_evs_new.iloc[i, :] = np.concatenate([tmp_left, tmp_right])

    # a few correction checks
    assert any(qis_evs_new.diff(axis=1).fillna(0) < 0), "Eigenvalues are not monotonically decreasing!"
    assert any(qis_evs_new.isna()), "There are NaN's in the QIS Eigenvalue Matrix!"
 
    return qis_evs_new

In [8]:
intensity_of_intensity_list = np.arange(0.5, 2.01, 0.05).round(2)

In [9]:
qis_evs_new = {}
intensities_new = {}
for intensity_of_intensity in intensity_of_intensity_list:
    qis_evs_new[intensity_of_intensity] = get_new_eigenvalues(evs_dfs["qis"], evs_dfs["sample"], intensity_of_intensity)

## Load return data and create training data for new qis eigenvalues

In [10]:
import pickle

base_folder_path = r'H:\\all\\RL_Shrinkage_2024'
# IMPORT SHRK DATASETS
pf_size = PF_SIZE  # DONT CHANGE HERE!!
permnos = pd.read_pickle(
    fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\permnos_1Y_p{pf_size}.pickle")
rets_full = pd.read_pickle(
    fr"{base_folder_path}\ONE_YR\preprocessing\rets_permnos_1Y\returns_full_1Y_p{pf_size}.pickle")

fixed_shrk_name = 'cov2Para'
opt_shrk_name = 'cov2Para'
with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\fixed_shrkges_cov2Para_p{pf_size}.pickle", 'rb') as f:
    fixed_shrk_data = pickle.load(f)
with open(rf"{base_folder_path}\ONE_YR\preprocessing\training_dfs\PF{pf_size}\cov2Para_factor-1.0_p{pf_size}.pickle", 'rb') as f:
    optimal_shrk_data = pickle.load(f)

# get all the validation indices
len_train = 5040
end_date = fixed_shrk_data.shape[0]
# temp here
val_indices_correct = (len_train, end_date)
val_indices_results = [val_indices_correct[0] + 21 * i for i in range((val_indices_correct[-1] - val_indices_correct[0]) // 21)]
val_idxes_shrkges = [0 + 21 * i for i in range((val_indices_correct[-1] - val_indices_correct[0]) // 21)]
reb_date_1 = permnos.index[0]
add_idx = np.where(rets_full.index == reb_date_1)[0][0]

In [11]:
try:
    ioi_path = r"H:\all\RL_Shrinkage_2024\ONE_YR\NonLinear_Shrinkage\intensity_of_intensity_data"
    all_res = pd.read_csv(ioi_path + f"\\all_res_p{PF_SIZE}_v2.csv", index_col=0)
    all_rawres = pd.read_csv(ioi_path + f"\\all_rawres_p{PF_SIZE}_v2.csv", index_col=0)
except:
    tmp_rawres, tmp_res =  get_rawres(qis_evs_new, modelnames = intensity_of_intensity_list)
    all_res = pd.DataFrame(tmp_res.copy())
    all_rawres = pd.DataFrame(tmp_rawres.copy())
    out_path = r"H:\all\RL_Shrinkage_2024\ONE_YR\NonLinear_Shrinkage\intensity_of_intensity_data"
    all_res.to_csv(out_path + f"\\all_res_p{PF_SIZE}_v2.csv")
    all_rawres.to_csv(out_path + f"\\all_rawres_p{PF_SIZE}_v2.csv")

done 0 out of 10353
done 250 out of 10353
done 500 out of 10353
done 750 out of 10353
done 1000 out of 10353
done 1250 out of 10353
done 1500 out of 10353
done 1750 out of 10353
done 2000 out of 10353
done 2250 out of 10353
done 2500 out of 10353
done 2750 out of 10353
done 3000 out of 10353
done 3250 out of 10353
done 3500 out of 10353
done 3750 out of 10353
done 4000 out of 10353
done 4250 out of 10353
done 4500 out of 10353
done 4750 out of 10353
done 5000 out of 10353
done 5250 out of 10353
done 5500 out of 10353
done 5750 out of 10353
done 6000 out of 10353
done 6250 out of 10353
done 6500 out of 10353
done 6750 out of 10353
done 7000 out of 10353
done 7250 out of 10353
done 7500 out of 10353
done 7750 out of 10353
done 8000 out of 10353
done 8250 out of 10353
done 8500 out of 10353
done 8750 out of 10353
done 9000 out of 10353
done 9250 out of 10353
done 9500 out of 10353
done 9750 out of 10353
done 10000 out of 10353
done 10250 out of 10353


In [ ]:
all_rawres.iloc[5040:, :].std()  * np.sqrt(252) * 100

In [ ]:
intensities={}
for int in qis_evs_new.keys():
    intensities[int] = qis_evs_new[int].iloc[:, -251:] / evs_dfs["sample"].iloc[:, -251:]


In [ ]:
allres_min_idxes_full = all_res.idxmin(axis=1)[: -21].values
allres_min_idxes_full = np.insert(allres_min_idxes_full, 0, np.repeat(["1.0"], 21))

# for sanity check: BIASED version should generally be better than
# non biased version as it is literally the minimum over the future 21 days
# so using it as a signal should outperform
allres_min_idxes_BIASED = all_res.idxmin(axis=1).values
allres_min_idxes_BIASED = allres_min_idxes_BIASED


# simple argmin rule, with full allres_min, should be same results as above
allres_min_idxes_full_v2 = allres_min_idxes_full[list(range(0, allres_min_idxes_full.shape[0], 21))]
allres_min_idxes_full_v2 = np.repeat(allres_min_idxes_full_v2, 21)
res_simple_argmin_rule = np.diag(all_rawres.loc[:, allres_min_idxes_full_v2])[5040:]


# simple argmin rule, biased (as a sanity check)
allres_min_idxes_BIASED_v2 = allres_min_idxes_BIASED[list(range(0, allres_min_idxes_BIASED.shape[0], 21))]
allres_min_idxes_BIASED_v2 = np.repeat(allres_min_idxes_BIASED_v2, 21)
res_simple_argmin_rule_biased = np.diag(all_rawres.loc[:, allres_min_idxes_BIASED_v2])[5040:]

res_actual_argmin = []
for i in range(5313//21):
    tmp_data = all_rawres.iloc[5040 + 21*i: 5040 + 21*(i+1)]
    curmin_idx = tmp_data.std().idxmin()
    curmin = tmp_data.loc[:, curmin_idx]
    res_actual_argmin += curmin.tolist()
# np.std(res_actual_argmin) * np.sqrt(252) * 100  --> 10.375

res_actual_argmin_nonbiased = []
for i in range(5313//21):
    idx_min_data = all_rawres.iloc[5040 - 21 + 21*i: 5040 - 21 + 21*(i+1)]
    curmin_idx = idx_min_data.std().idxmin()
    tmp_data = all_rawres.iloc[5040 + 21*i: 5040 + 21*(i+1)]
    curmin = tmp_data.loc[:, curmin_idx]
    res_actual_argmin_nonbiased += curmin.tolist()
#np.std(res_actual_argmin_nonbiased) * np.sqrt(252) * 100  --> 10.65

#min_idxes = fixed_shrk_data.iloc[:, 3:].idxmin(axis=1)
#opt_vals = np.diag(fixed_shrk_data.iloc[:, 3:].loc[:, min_idxes])

# get all the validation indices
len_train = 5040
end_date = fixed_shrk_data.shape[0]
# temp here
val_indices_correct = (len_train, end_date)
val_indices_results = [val_indices_correct[0] + 21 * i for i in range((val_indices_correct[-1] - val_indices_correct[0]) // 21)]
val_idxes_shrkges = [0 + 21 * i for i in range((val_indices_correct[-1] - val_indices_correct[0]) // 21)]
reb_date_1 = permnos.index[0]
add_idx = np.where(rets_full.index == reb_date_1)[0][0]

In [ ]:
# change path
os.chdir(r'H:\all\RL_Shrinkage_2024')
from ONE_YR.NonLinear_Shrinkage import regression_evaluation_funcs as re_hf

In [ ]:
all_factors = intensity_of_intensity_list
Y = allres_min_idxes_BIASED.astype(float)
opt_values = allres_min_idxes_BIASED.astype(float)[:-21]
opt_values = np.insert(arr=opt_values, obj=0, values=np.repeat(0.0, 21))

Y = np.array(re_hf.map_factors_to_preds(Y.reshape(-1), all_factors))


In [ ]:
def map_factors_to_preds(preds_as_floats, all_factors):
    all_nums = np.arange(len(all_factors))
    mapping = {}
    for i, f in enumerate(all_factors):
        mapping[f] = all_nums[i]
    model_preds = list(map(lambda x: mapping[x], preds_as_floats))
    return model_preds

In [ ]:
all_factors = intensity_of_intensity_list
Y = allres_min_idxes_BIASED.astype(float)
opt_values = allres_min_idxes_BIASED.astype(float)[:-21]
opt_values = np.insert(arr=opt_values, obj=0, values=np.repeat(1.0, 21))
Y = np.array(re_hf.map_factors_to_preds(Y.reshape(-1), all_factors))
opt_values = np.array(re_hf.map_factors_to_preds(opt_values, all_factors))
opt_v3 = np.diag(all_res.loc[:, allres_min_idxes_BIASED])[:-21]
opt_v3 = np.insert(arr=opt_v3, obj=0, values=np.repeat(7.0, 21))

#opt_v3 = pd.Series(opt_v3).rolling(window=40, min_periods=1).mean()

In [ ]:
opt_v4 = np.copy(opt_v3)
opt_v4 = np.insert(arr=opt_v3, obj=0, values=np.repeat(7.0, 21))[:-21]

rolling_opt = pd.Series(opt_values).rolling(window=252, min_periods=1).mean().values
rolling_opt2 = pd.Series(opt_v3).rolling(window=252, min_periods=1).mean().values

In [ ]:
params = {
    'pf_size' : pf_size,
    'opt_values_factors' : opt_values,
'include_ts_momentum_var_allstocks': False,
'include_ts_momentum_allstocks': True,
'include_sample_covmat_trace': True,
'include_mean_of_correls': False,
'include_iqr': False,
'include_factors': False,
'include_ewma_year': False,
'include_ewma_month': True,
'include_ew_year_vola': False,
'include_ew_month_vola': True,
'include_allstocks_year_avgvola': True,
'include_allstocks_month_avgvola': False,
    'additional_inputs' : [opt_v3]  # opt_v4, rolling_opt, rolling_opt2
}

X = re_hf.load_additional_train_data(**params)

In [ ]:
def eval_model_outputs(res, intensity_of_intensity_list, show_fig=True):
    res = [len(intensity_of_intensity_list)-1 if r > len(intensity_of_intensity_list) else r for r in res]
    res = [0 if r < 0 else r for r in res]
    res = re_hf.map_preds_to_factors(res, all_factors)
    Y_eval = all_rawres
    Y_eval.columns = Y_eval.columns.astype(str)
    res_evaluated2 = re_hf.evaluate_all_factor_preds(res, Y_eval, len_train)
    datetime_index = pd.to_datetime(permnos.index, format="%Y%m%d")
    res_df = pd.DataFrame(np.array(res, dtype=float), index=datetime_index[5040:], columns=[""])
    
    fig = px.line(res_df, height=500, width=1000).update_layout(
        xaxis_title="Year", yaxis_title="Intensity of Intensity")
    
    print("Result:", res_evaluated2)
    if show_fig:
        fig.show()

    return fig

In [ ]:
%%capture
res = re_hf.general_single_output_ElasticNet_Lagged(X=X, Y=Y, len_train=5040)

In [ ]:
fig = eval_model_outputs(res, intensity_of_intensity_list, show_fig=True)

In [ ]:
all_rawres.iloc[5040:, :].std() * np.sqrt(252) * 100